In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import math
from collections import Counter
import math

In [2]:
data = pd.read_csv("https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv")
data.to_excel("iris.xlsx",index=False,sheet_name='sheet_1')
iris_df = pd.read_excel('iris.xlsx')

In [3]:
iris_df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [4]:
data_latih, data_uji = train_test_split(data,test_size=0.2)
label_uji = data_uji.pop('variety')

In [5]:
print(data_latih.shape[0])
print(data_uji.shape[0])
data_uji.head()

120
30


,sepal.length,sepal.width,petal.length,petal.width
3,4.6,3.1,1.5,0.2
92,5.8,2.6,4.0,1.2
18,5.7,3.8,1.7,0.3
116,6.5,3.0,5.5,1.8
106,4.9,2.5,4.5,1.7


In [6]:
def hitung_prior(list_kelas):
    n_data = len(list_kelas)
    prior = Counter(list_kelas)
    for i in prior.keys():
        prior[i]=prior[i]/n_data
    return prior

In [7]:
prior = hitung_prior(data_latih['variety'])
print(prior)

Counter({'Virginica': 0.36666666666666664, 'Versicolor': 0.31666666666666665, 'Setosa': 0.31666666666666665})


In [8]:
def hitung_std_kelas(input_data):
    list_columns = input_data.columns[:-1]
    class_column = input_data.columns[-1]
    list_kelas = set(input_data[class_column])
    rata2 = {}
    std = {}
    for column in list_columns:
        for a_class in list_kelas:
            rata2[(a_class,column)] = input_data.loc[input_data[class_column]==a_class][column].mean()
            std[a_class,column] = input_data.loc[input_data[class_column]==a_class][column].std()
    return(rata2,std)

In [9]:
def hitung_likelihood_gaussian(data,rata2,std):
    hasil = (1/math.sqrt(2*math.pi*(std**2)))*math.exp((-1*((data-rata2)**2))/(2*(std**2)))
    return hasil

In [10]:
# column
print(data_latih.columns[:-1])
# a_class
print(set(data_latih[data_latih.columns[-1]]))
# class_column
print(data_latih.columns[-1])
# isian Setosa
print(data_latih.loc[data_latih["variety"]=="Setosa"][data_latih.columns[:-1]].mean())

Index(['sepal.length', 'sepal.width', 'petal.length', 'petal.width'], dtype='object')
{'Versicolor', 'Virginica', 'Setosa'}
variety
sepal.length    4.965789
sepal.width     3.392105
petal.length    1.455263
petal.width     0.252632
dtype: float64


### Training Gaussian

In [11]:
def training_naive_bayes_gaussian(data_latih):
    class_column = data_latih.columns[-1]
    list_columns = data_latih.columns[:-1]
    prior = hitung_prior(data_latih[class_column])
    (rata2,std) = hitung_std_kelas(data_latih)
    list_class = set(data_latih[class_column])
    model = {}
    model['prior'] = prior
    model['rata2'] = rata2
    model['std'] = std
    model['list_class'] = list_class
    model['list_columns'] = list_columns
    return model

In [17]:
def testing_naive_bayes_gaussian(model,data_uji):
    prior = model['prior']
    rata2 = model['rata2']
    std = model['std']
    list_class = model['list_class']
    list_columns = model['list_columns']
    posterior = dict.fromkeys(list_class,1)
    for a_class in list_class:
        for column in list_columns:
            posterior[a_class] = posterior[a_class] * hitung_likelihood_gaussian(data_uji[column],rata2[(a_class,column)],std[(a_class,column)])
        posterior[a_class] = posterior[a_class]*prior[a_class]
    kelas_uji = max(posterior,key=posterior.get)
    return kelas_uji


In [18]:
index_uji = 0 #data ke-0
model = training_naive_bayes_gaussian(data_latih)
prediksi = testing_naive_bayes_gaussian(model,data_uji.iloc[index_uji])
print(prediksi)
print(label_uji.iloc[index_uji])

Setosa
Setosa


In [34]:
#menguji semua datanya
all_prediction = []
for index in range(data_uji.shape[0]):
    all_prediction.append(testing_naive_bayes_gaussian(model,data_uji.iloc[index]))

prediksi = pd.Series(all_prediction)
label_uji_reset = pd.Series(label_uji).reset_index(drop=True)
combined_series = pd.concat([prediksi, label_uji_reset], axis=1)
combined_series.columns = ["Prediksi","Kenyataan"]
print(combined_series)

      Prediksi   Kenyataan
0       Setosa      Setosa
1   Versicolor  Versicolor
2       Setosa      Setosa
3    Virginica   Virginica
4   Versicolor   Virginica
5       Setosa      Setosa
6   Versicolor  Versicolor
7    Virginica   Virginica
8   Versicolor  Versicolor
9   Versicolor   Virginica
10  Versicolor  Versicolor
11      Setosa      Setosa
12   Virginica   Virginica
13   Virginica   Virginica
14      Setosa      Setosa
15      Setosa      Setosa
16  Versicolor  Versicolor
17      Setosa      Setosa
18  Versicolor  Versicolor
19  Versicolor  Versicolor
20      Setosa      Setosa
21  Versicolor  Versicolor
22      Setosa      Setosa
23  Versicolor  Versicolor
24  Versicolor  Versicolor
25      Setosa      Setosa
26  Versicolor  Versicolor
27      Setosa      Setosa
28  Versicolor  Versicolor
29      Setosa      Setosa


In [40]:
def hasil(hasil_uji, label_uji):
    count = 0
    for i,label in enumerate(label_uji):
        if label != hasil_uji[i]:
            count = count + 1
    return count

test = hasil(all_prediction,label_uji)
print(f"hasil yang berbeda ada {test}")

hasil yang berbeda ada 2
